## 7. MNIST를 위한 고성능 CNN

자신만의 CNN을 만들고 MNIST 데이터셋에서 가능한 최대 정확도를 달성해보세요.

여기에서 11장의 내용을 적용하자. 드롭아웃과 미니배치 정규화

전이학습은 9번에서 하니까 여기서에는 위의 두 가지만

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_sample_image
import matplotlib.pyplot as plt

/Users/woohyunhan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
height = 28
width = 28
channels = 1
n_inputs = height * width

In [24]:
tf.reset_default_graph()

In [25]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape = [None, n_inputs], name = "X")
    X_reshaped = tf.reshape(X, shape = [-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name = "y")
    training = tf.placeholder_with_default(False, shape = [], name = "training")

In [26]:
conv1 = tf.layers.conv2d(X_reshaped,
                        filters = 6,
                        kernel_size = 5,
                        strides = [1,1],
                        padding = "SAME",
                        activation = tf.nn.tanh,
                        name = "conv1")

In [27]:
pool1 = tf.nn.avg_pool(conv1, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1],
                       padding = "VALID", name = "pool1")
pool1_act = tf.nn.tanh(pool1)

In [28]:
conv2 = tf.layers.conv2d(pool1_act,
                        filters = 16,
                        kernel_size = 5,
                        strides = [1,1],
                        padding = "VALID",
                        activation = tf.nn.tanh,
                        name = "conv2")

In [29]:
pool2 = tf.nn.avg_pool(conv2, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1],
                       padding = "VALID", name = "pool2")
pool2_act = tf.nn.tanh(pool2)

In [30]:
conv3 = tf.layers.conv2d(pool2_act,
                        filters = 120,
                        kernel_size = 5,
                        strides = [1,1],
                        padding = "VALID",
                        activation = tf.nn.tanh,
                        name = "conv3")

In [31]:
he_init = tf.variance_scaling_initializer()

conv3_flat = tf.reshape(conv3, shape = [-1, 120])

dnn1 = tf.layers.dense(conv3_flat, 84, activation= tf.nn.tanh,
                      name = "Dense1")

logits = tf.layers.dense(dnn1, 10, activation= tf.nn.tanh,
                      name = "Dense2")

Y_proba = tf.nn.softmax(logits, name = "Y_proba")

In [32]:
global_step = tf.Variable(initial_value=0, trainable= False,
                         name= "global_step")

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits= logits,
                                                         labels= y)

loss = tf.reduce_mean(xentropy,name= "loss")
learning_rate = 0.001 
optimizer = tf.train.AdadeltaOptimizer(learning_rate)
training_op = optimizer.minimize(loss, global_step= global_step)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [48]:
def shuffle_batch(X_train, y_train, batch_size):
    rnd_idx = np.random.permutation(len(X_train))
    n_batches = len(X_train) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
        #print(X_batch.shape,y_batch.shape)
        yield X_batch, y_batch

In [49]:
from datetime import datetime
loss_summary = tf.summary.scalar("Loss", loss)
now = datetime.now().strftime("%Y%m%d_%H-%M-%S")
log_dir = "Exercises7_log/run-{}".format(now)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [50]:
X_train = mnist.train.images
X_test = mnist.test.images
X_valid = mnist.validation.images
y_train = mnist.train.labels
y_test = mnist.test.labels
y_valid = mnist.validation.labels
X_train.shape

(55000, 784)

In [ ]:
n_epochs = 10
batch_size = 4

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict= {X: X_batch, y: y_batch})
            
            summary_val = loss_summary.eval(feed_dict= {X: X_batch, y: y_batch})
            file_writer.add_summary(summary_val, tf.train.global_step(sess, global_step))
        
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도: ", acc_batch, 
              "\t검증 세트 정확도: ", acc_valid)
    
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("테스트 세트에서의 최종 정확도: ", round(acc_test,2))
    
    save_path = saver.save(sess, "./my_mnist_model")